In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import requests
import re
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
import os
import pandas as pd
import requests
import re
import numpy as np

############ Functions to fetch data ############################

def clean_doi(doi):
    """
    Function to transform DOIs to the required format.
    Converts https://doi.org/10.21203/rs.3.pex-2537/v1 to 10.21203/rs.3.pex-2537/v1.
    """
    if doi and doi.startswith("https://doi.org/"):
        return doi.replace("https://doi.org/", "")
    return doi

def reconstruct_abstract(abstract_inverted_index):
    """
    Reconstructs the abstract from the abstract_inverted_index.
    """
    if not abstract_inverted_index:
        return np.nan

    # Create a dictionary to hold position-word mappings
    abstract_dict = {}

    # Iterate over keys (which are words) and positions
    for word, positions in abstract_inverted_index.items():
        for pos in positions:
            abstract_dict[pos] = word

    # Sort by the position and join the words in order
    sorted_positions = sorted(abstract_dict.keys())
    abstract_words = [abstract_dict[pos] for pos in sorted_positions]

    return " ".join(abstract_words)

def fetch_openalex_publications(search_query, max_results=100000):
    works = []
    per_page = 25
    cursor = '*'
    while True:
        url = f"{search_query}&per-page={per_page}&cursor={cursor}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            results = data.get('results', [])
            if not results:
                break
            works.extend(results)
            if len(works) >= max_results:
                break
            cursor = data.get('meta', {}).get('next_cursor')
            if not cursor:
                break
        except requests.RequestException as e:
            print(f"Error fetching OpenAlex publications: {e}")
            break
    return works

def fetch_crossref_metadata(doi):
    doi = clean_doi(doi)
    url = f'https://api.crossref.org/works/{doi}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json().get('message', {})
    except requests.RequestException as e:
        print(f"Error fetching Crossref metadata: {e}")
        return {}

def fetch_altmetric_data(doi):
    doi = clean_doi(doi)
    altmetric_url = f'https://api.altmetric.com/v1/doi/{doi}'
    try:
        response = requests.get(altmetric_url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching Altmetric data: {e}")
        return {}

def build_publications_dataframe(search_query, include_crossref, include_altmetric):
    works = fetch_openalex_publications(search_query)
    publications_data = [collect_publication_info(work, include_crossref, include_altmetric) for work in works]
    df = pd.DataFrame(publications_data)

    # Country code to full name and region mapping
    country_mapping = {
    "US": {"name": "United States", "region": "North America", "income": "High income"},
    "KR": {"name": "South Korea", "region": "Asia", "income": "High income"},
    "AU": {"name": "Australia", "region": "Oceania", "income": "High income"},
    "HK": {"name": "Hong Kong", "region": "Asia", "income": "High income"},
    "FR": {"name": "France", "region": "Europe", "income": "High income"},
    "BR": {"name": "Brazil", "region": "South America", "income": "Upper middle income"},
    "LT": {"name": "Lithuania", "region": "Europe", "income": "High income"},
    "CA": {"name": "Canada", "region": "North America", "income": "High income"},
    "DE": {"name": "Germany", "region": "Europe", "income": "High income"},
    "IN": {"name": "India", "region": "Asia", "income": "Lower middle income"},
    "CN": {"name": "China", "region": "Asia", "income": "Upper middle income"},
    "GB": {"name": "United Kingdom", "region": "Europe", "income": "High income"},
    "JP": {"name": "Japan", "region": "Asia", "income": "High income"},
    "ZA": {"name": "South Africa", "region": "Africa", "income": "Upper middle income"},
    "NG": {"name": "Nigeria", "region": "Africa", "income": "Lower middle income"},
    "MX": {"name": "Mexico", "region": "North America", "income": "Upper middle income"},
    "RU": {"name": "Russia", "region": "Europe", "income": "Upper middle income"},
    "IT": {"name": "Italy", "region": "Europe", "income": "High income"},
    "ES": {"name": "Spain", "region": "Europe", "income": "High income"},
    "KE": {"name": "Kenya", "region": "Africa", "income": "Lower middle income"},
    "EG": {"name": "Egypt", "region": "Africa", "income": "Lower middle income"},
    "AR": {"name": "Argentina", "region": "South America", "income": "Upper middle income"},
    "CL": {"name": "Chile", "region": "South America", "income": "High income"},
    "CO": {"name": "Colombia", "region": "South America", "income": "Upper middle income"},
    "SA": {"name": "Saudi Arabia", "region": "Asia", "income": "High income"},
    "NZ": {"name": "New Zealand", "region": "Oceania", "income": "High income"},
    "SE": {"name": "Sweden", "region": "Europe", "income": "High income"},
    "NO": {"name": "Norway", "region": "Europe", "income": "High income"},
    "FI": {"name": "Finland", "region": "Europe", "income": "High income"},
    "DK": {"name": "Denmark", "region": "Europe", "income": "High income"},
    "TR": {"name": "Turkey", "region": "Asia", "income": "Upper middle income"},
    "PL": {"name": "Poland", "region": "Europe", "income": "High income"},
    "PT": {"name": "Portugal", "region": "Europe", "income": "High income"},
    "ID": {"name": "Indonesia", "region": "Asia", "income": "Lower middle income"},
    "IR": {"name": "Iran", "region": "Asia", "income": "Upper middle income"},
    "TH": {"name": "Thailand", "region": "Asia", "income": "Upper middle income"},
    "PH": {"name": "Philippines", "region": "Asia", "income": "Lower middle income"},
    "MY": {"name": "Malaysia", "region": "Asia", "income": "Upper middle income"},
    "BD": {"name": "Bangladesh", "region": "Asia", "income": "Lower middle income"},
    "PK": {"name": "Pakistan", "region": "Asia", "income": "Lower middle income"},
    "VN": {"name": "Vietnam", "region": "Asia", "income": "Lower middle income"},
    "ET": {"name": "Ethiopia", "region": "Africa", "income": "Low income"},
    "GH": {"name": "Ghana", "region": "Africa", "income": "Lower middle income"},
    "UG": {"name": "Uganda", "region": "Africa", "income": "Low income"},
    "TZ": {"name": "Tanzania", "region": "Africa", "income": "Low income"},
    "MO": {"name": "Macao", "region": "Asia", "income": "High income"},
    "AE": {"name": "United Arab Emirates", "region": "Asia", "income": "High income"},
    "QA": {"name": "Qatar", "region": "Asia", "income": "High income"},
    "BH": {"name": "Bahrain", "region": "Asia", "income": "High income"},
    "JO": {"name": "Jordan", "region": "Asia", "income": "Upper middle income"},
    "LB": {"name": "Lebanon", "region": "Asia", "income": "Upper middle income"},
    "CY": {"name": "Cyprus", "region": "Asia", "income": "High income"},
    "GR": {"name": "Greece", "region": "Europe", "income": "High income"},
    "HR": {"name": "Croatia", "region": "Europe", "income": "High income"},
    "RO": {"name": "Romania", "region": "Europe", "income": "High income"},
    "SI": {"name": "Slovenia", "region": "Europe", "income": "High income"},
    "SK": {"name": "Slovakia", "region": "Europe", "income": "High income"},
    "BG": {"name": "Bulgaria", "region": "Europe", "income": "Upper middle income"},
    "MK": {"name": "North Macedonia", "region": "Europe", "income": "Upper middle income"},
    "AL": {"name": "Albania", "region": "Europe", "income": "Upper middle income"},
    "MD": {"name": "Moldova", "region": "Europe", "income": "Upper middle income"},
    "UA": {"name": "Ukraine", "region": "Europe", "income": "Upper middle income"},

    "AF": {"name": "Afghanistan", "region": "Asia", "income": "Low income"},
    "AL": {"name": "Albania", "region": "Europe", "income": "Upper middle income"},
    "DZ": {"name": "Algeria", "region": "Africa", "income": "Upper middle income"},
    "AD": {"name": "Andorra", "region": "Europe", "income": "High income"},
    "AO": {"name": "Angola", "region": "Africa", "income": "Lower middle income"},
    "AG": {"name": "Antigua and Barbuda", "region": "North America", "income": "High income"},
    "AM": {"name": "Armenia", "region": "Asia", "income": "Upper middle income"},
    "AZ": {"name": "Azerbaijan", "region": "Asia", "income": "Upper middle income"},
    "BS": {"name": "Bahamas", "region": "North America", "income": "High income"},
    "BB": {"name": "Barbados", "region": "North America", "income": "High income"},
    "BY": {"name": "Belarus", "region": "Europe", "income": "Upper middle income"},
    "BZ": {"name": "Belize", "region": "North America", "income": "Upper middle income"},
    "BJ": {"name": "Benin", "region": "Africa", "income": "Lower middle income"},
    "BT": {"name": "Bhutan", "region": "Asia", "income": "Lower middle income"},
    "BO": {"name": "Bolivia", "region": "South America", "income": "Lower middle income"},
    "BA": {"name": "Bosnia and Herzegovina", "region": "Europe", "income": "Upper middle income"},
    "BW": {"name": "Botswana", "region": "Africa", "income": "Upper middle income"},
    "BN": {"name": "Brunei", "region": "Asia", "income": "High income"},
    "BF": {"name": "Burkina Faso", "region": "Africa", "income": "Low income"},
    "BI": {"name": "Burundi", "region": "Africa", "income": "Low income"},
    "CV": {"name": "Cabo Verde", "region": "Africa", "income": "Lower middle income"},
    "KH": {"name": "Cambodia", "region": "Asia", "income": "Lower middle income"},
    "CM": {"name": "Cameroon", "region": "Africa", "income": "Lower middle income"},
    "CF": {"name": "Central African Republic", "region": "Africa", "income": "Low income"},
    "TD": {"name": "Chad", "region": "Africa", "income": "Low income"},
    "KM": {"name": "Comoros", "region": "Africa", "income": "Lower middle income"},
    "CD": {"name": "Congo, Democratic Republic of the", "region": "Africa", "income": "Low income"},
    "CG": {"name": "Congo, Republic of the", "region": "Africa", "income": "Lower middle income"},
    "CR": {"name": "Costa Rica", "region": "North America", "income": "Upper middle income"},
    "CU": {"name": "Cuba", "region": "North America", "income": "Upper middle income"},
    "DJ": {"name": "Djibouti", "region": "Africa", "income": "Lower middle income"},
    "DM": {"name": "Dominica", "region": "North America", "income": "Upper middle income"},
    "DO": {"name": "Dominican Republic", "region": "North America", "income": "Upper middle income"},
    "TL": {"name": "East Timor (Timor-Leste)", "region": "Asia", "income": "Lower middle income"},
    "EC": {"name": "Ecuador", "region": "South America", "income": "Upper middle income"},
    "SV": {"name": "El Salvador", "region": "North America", "income": "Lower middle income"},
    "GQ": {"name": "Equatorial Guinea", "region": "Africa", "income": "Upper middle income"},
    "ER": {"name": "Eritrea", "region": "Africa", "income": "Low income"},
    "SZ": {"name": "Eswatini", "region": "Africa", "income": "Lower middle income"},
    "FJ": {"name": "Fiji", "region": "Oceania", "income": "Upper middle income"},
    "GA": {"name": "Gabon", "region": "Africa", "income": "Upper middle income"},
    "GM": {"name": "Gambia", "region": "Africa", "income": "Lower middle income"},
    "GE": {"name": "Georgia", "region": "Asia", "income": "Upper middle income"},
    "GD": {"name": "Grenada", "region": "North America", "income": "High income"},
    "GT": {"name": "Guatemala", "region": "North America", "income": "Upper middle income"},
    "GN": {"name": "Guinea", "region": "Africa", "income": "Low income"},
    "GW": {"name": "Guinea-Bissau", "region": "Africa", "income": "Low income"},
    "GY": {"name": "Guyana", "region": "South America", "income": "Upper middle income"},
    "HT": {"name": "Haiti", "region": "North America", "income": "Low income"},
    "HN": {"name": "Honduras", "region": "North America", "income": "Lower middle income"},
    "IS": {"name": "Iceland", "region": "Europe", "income": "High income"},
    "IQ": {"name": "Iraq", "region": "Asia", "income": "Upper middle income"},
    "IE": {"name": "Ireland", "region": "Europe", "income": "High income"},
    "IL": {"name": "Israel", "region": "Asia", "income": "High income"},
    "CI": {"name": "Ivory Coast (Côte d'Ivoire)", "region": "Africa", "income": "Lower middle income"},
    "JM": {"name": "Jamaica", "region": "North America", "income": "Upper middle income"},
    "KZ": {"name": "Kazakhstan", "region": "Asia", "income": "Upper middle income"},
    "KI": {"name": "Kiribati", "region": "Oceania", "income": "Lower middle income"},
    "XK": {"name": "Kosovo", "region": "Europe", "income": "Upper middle income"},
    "KW": {"name": "Kuwait", "region": "Asia", "income": "High income"},
    "KG": {"name": "Kyrgyzstan", "region": "Asia", "income": "Lower middle income"},
    "LA": {"name": "Laos", "region": "Asia", "income": "Lower middle income"},
    "LV": {"name": "Latvia", "region": "Europe", "income": "High income"},
    "LS": {"name": "Lesotho", "region": "Africa", "income": "Lower middle income"},
    "LR": {"name": "Liberia", "region": "Africa", "income": "Low income"},
    "LY": {"name": "Libya", "region": "Africa", "income": "Upper middle income"},
    "LI": {"name": "Liechtenstein", "region": "Europe", "income": "High income"},
    "LU": {"name": "Luxembourg", "region": "Europe", "income": "High income"},
    "MG": {"name": "Madagascar", "region": "Africa", "income": "Low income"},
    "MW": {"name": "Malawi", "region": "Africa", "income": "Low income"},
    "MV": {"name": "Maldives", "region": "Asia", "income": "Upper middle income"},
    "ML": {"name": "Mali", "region": "Africa", "income": "Low income"},
    "MT": {"name": "Malta", "region": "Europe", "income": "High income"},
    "MH": {"name": "Marshall Islands", "region": "Oceania", "income": "Upper middle income"},
    "MR": {"name": "Mauritania", "region": "Africa", "income": "Lower middle income"},
    "MU": {"name": "Mauritius", "region": "Africa", "income": "Upper middle income"},
    "FM": {"name": "Micronesia", "region": "Oceania", "income": "Lower middle income"},
    "MC": {"name": "Monaco", "region": "Europe", "income": "High income"},

    "MN": {"name": "Mongolia", "region": "Asia", "income": "Lower middle income"},
    "ME": {"name": "Montenegro", "region": "Europe", "income": "Upper middle income"},
    "MA": {"name": "Morocco", "region": "Africa", "income": "Lower middle income"},
    "MZ": {"name": "Mozambique", "region": "Africa", "income": "Low income"},
    "NA": {"name": "Namibia", "region": "Africa", "income": "Upper middle income"},
    "NR": {"name": "Nauru", "region": "Oceania", "income": "High income"},
    "NP": {"name": "Nepal", "region": "Asia", "income": "Lower middle income"},
    "NI": {"name": "Nicaragua", "region": "North America", "income": "Lower middle income"},
    "NE": {"name": "Niger", "region": "Africa", "income": "Low income"},
    "KP": {"name": "North Korea", "region": "Asia", "income": "Low income"},
    "MK": {"name": "North Macedonia", "region": "Europe", "income": "Upper middle income"},
    "OM": {"name": "Oman", "region": "Asia", "income": "High income"},
    "PW": {"name": "Palau", "region": "Oceania", "income": "High income"},
    "PA": {"name": "Panama", "region": "North America", "income": "Upper middle income"},
    "PG": {"name": "Papua New Guinea", "region": "Oceania", "income": "Lower middle income"},
    "PY": {"name": "Paraguay", "region": "South America", "income": "Upper middle income"},
    "PE": {"name": "Peru", "region": "South America", "income": "Upper middle income"},
    "RW": {"name": "Rwanda", "region": "Africa", "income": "Low income"},
    "KN": {"name": "Saint Kitts and Nevis", "region": "North America", "income": "High income"},
    "LC": {"name": "Saint Lucia", "region": "North America", "income": "Upper middle income"},
    "VC": {"name": "Saint Vincent and the Grenadines", "region": "North America", "income": "Upper middle income"},
    "WS": {"name": "Samoa", "region": "Oceania", "income": "Lower middle income"},
    "SM": {"name": "San Marino", "region": "Europe", "income": "High income"},
    "ST": {"name": "Sao Tome and Principe", "region": "Africa", "income": "Lower middle income"},
    "SN": {"name": "Senegal", "region": "Africa", "income": "Lower middle income"},
    "SC": {"name": "Seychelles", "region": "Africa", "income": "High income"},
    "SL": {"name": "Sierra Leone", "region": "Africa", "income": "Low income"},
    "SG": {"name": "Singapore", "region": "Asia", "income": "High income"},
    "SB": {"name": "Solomon Islands", "region": "Oceania", "income": "Lower middle income"},
    "SO": {"name": "Somalia", "region": "Africa", "income": "Low income"},
    "SS": {"name": "South Sudan", "region": "Africa", "income": "Low income"},
    "LK": {"name": "Sri Lanka", "region": "Asia", "income": "Lower middle income"},
    "SD": {"name": "Sudan", "region": "Africa", "income": "Lower middle income"},
    "SR": {"name": "Suriname", "region": "South America", "income": "Upper middle income"},
    "SY": {"name": "Syria", "region": "Asia", "income": "Low income"},
    "TJ": {"name": "Tajikistan", "region": "Asia", "income": "Lower middle income"},
    "TO": {"name": "Tonga", "region": "Oceania", "income": "Upper middle income"},
    "TT": {"name": "Trinidad and Tobago", "region": "North America", "income": "High income"},
    "TN": {"name": "Tunisia", "region": "Africa", "income": "Lower middle income"},
    "TM": {"name": "Turkmenistan", "region": "Asia", "income": "Upper middle income"},
    "TV": {"name": "Tuvalu", "region": "Oceania", "income": "Upper middle income"},
    "UY": {"name": "Uruguay", "region": "South America", "income": "High income"},
    "UZ": {"name": "Uzbekistan", "region": "Asia", "income": "Lower middle income"},
    "VU": {"name": "Vanuatu", "region": "Oceania", "income": "Lower middle income"},
    "VE": {"name": "Venezuela", "region": "South America", "income": "Lower middle income"},
    "YE": {"name": "Yemen", "region": "Asia", "income": "Low income"},
    "ZM": {"name": "Zambia", "region": "Africa", "income": "Lower middle income"},
    "ZW": {"name": "Zimbabwe", "region": "Africa", "income": "Low income"},
    "TW": {"name": "Taiwan", "region": "Asia", "income": "High income"},
    "AT": {"name": "Austria", "region": "Europe", "income": "High income"},
    "AE": {"name": "United Arab Emirates", "region": "Asia", "income": "High income"},
    "BE": {"name": "Belgium", "region": "Europe", "income": "High income"},
    "BG": {"name": "Bulgaria", "region": "Europe", "income": "Upper middle income"},
    "CH": {"name": "Switzerland", "region": "Europe", "income": "High income"},
    "EG": {"name": "Egypt", "region": "Africa", "income": "Lower middle income"},
    "HU": {"name": "Hungary", "region": "Europe", "income": "High income"},
    "NL": {"name": "Netherlands", "region": "Europe", "income": "High income"},
    "PS": {"name": "Palestine", "region": "Asia", "income": "Lower middle income"},
    "RS": {"name": "Serbia", "region": "Europe", "income": "Upper middle income"},
    "CZ": {"name": "Czech Republic", "region": "Europe", "income": "High income"},
    "EE": {"name": "Estonia", "region": "Europe", "income": "High income"},
    "GL": {"name": "Greenland", "region": "North America", "income": "High income"},
    "GP": {"name": "Guadeloupe", "region": "North America", "income": "High income"},
    "IM": {"name": "Isle of Man", "region": "Europe", "income": "High income"},
    "JM": {"name": "Jamaica", "region": "North America", "income": "Upper middle income"},
    "LS": {"name": "Lesotho", "region": "Africa", "income": "Lower middle income"},
    "MF": {"name": "Saint Martin", "region": "North America", "income": "High income"},
    "MM": {"name": "Myanmar", "region": "Asia", "income": "Lower middle income"},
    "MQ": {"name": "Martinique", "region": "North America", "income": "High income"},
    "NE": {"name": "Niger", "region": "Africa", "income": "Low income"},
    "NI": {"name": "Nicaragua", "region": "North America", "income": "Lower middle income"},
    "PF": {"name": "French Polynesia", "region": "Oceania", "income": "Upper middle income"},
    "PR": {"name": "Puerto Rico", "region": "North America", "income": "High income"},
    "RE": {"name": "Réunion", "region": "Africa", "income": "Upper middle income"},
    "TG": {"name": "Togo", "region": "Africa", "income": "Low income"}
    }


    # Replace country codes with full names, regions, and incomes for relevant columns
    country_columns = [
        'OpenAlex Countries',
        'OpenAlex First Authors Countries',
        'OpenAlex Last Authors Countries',
        'OpenAlex Corresponding Authors Countries'
    ]

    for column in country_columns:
        df[f'{column} Full Name'] = df[column].apply(lambda x: np.nan if pd.isna(x) else ', '.join([str(country_mapping.get(str(code), {}).get("name", '')) for code in str(x).split(', ')]))
        df[f'{column} Region'] = df[column].apply(lambda x: np.nan if pd.isna(x) else ', '.join([str(country_mapping.get(str(code), {}).get("region", '')) for code in str(x).split(', ')]))
        df[f'{column} Income'] = df[column].apply(lambda x: np.nan if pd.isna(x) else ', '.join([str(country_mapping.get(str(code), {}).get("income", '')) for code in str(x).split(', ')]))

    return df

def collect_publication_info(work, include_crossref, include_altmetric):
    doi = work.get('doi', np.nan)
    crossref_metadata = fetch_crossref_metadata(doi) if include_crossref and pd.notna(doi) else {}
    altmetric_data = fetch_altmetric_data(doi) if include_altmetric and pd.notna(doi) else {}

    # Reconstruct the abstract if available
    abstract_inverted_index = work.get('abstract_inverted_index')
    abstract = reconstruct_abstract(abstract_inverted_index) if abstract_inverted_index else np.nan

    institutions_country = []
    institution_type = []
    first_authors_institutions_countries = []
    last_authors_institutions_countries = []
    corresponding_authors_institutions_countries = []

    for authorship in work.get('authorships', []):
        for institution in authorship.get('institutions', []):
            institution_name = institution.get('display_name')
            country_code = institution.get('country_code')
            type_of_institution = institution.get('type')
            institutions_country.append(f"{institution_name} ({country_code})")
            institution_type.append(f"{institution_name} ({type_of_institution})")
            if authorship.get('author_position') == 'first':
                first_authors_institutions_countries.append(f"{institution_name} ({country_code})")
            elif authorship.get('author_position') == 'last':
                last_authors_institutions_countries.append(f"{institution_name} ({country_code})")
            elif authorship.get('is_corresponding') == 'True':
                corresponding_authors_institutions_countries.append(f"{institution_name} ({country_code})")

    institutions_country = list(set(institutions_country))
    institution_type = list(set(institution_type))
    first_authors_institutions_countries = list(set(first_authors_institutions_countries))
    last_authors_institutions_countries = list(set(last_authors_institutions_countries))
    corresponding_authors_institutions_countries = list(set(corresponding_authors_institutions_countries))

    author_countries = set()
    for authorship in work.get('authorships', []):
        for country in authorship.get('countries', []):
            author_countries.add(country)
    author_countries = list(author_countries)

    first_authorships = [authorship for authorship in work.get('authorships', []) if authorship.get('author_position') == 'first']
    last_authorships = [authorship for authorship in work.get('authorships', []) if authorship.get('author_position') == 'last']
    corresponding_authorships = [authorship for authorship in work.get('authorships', []) if authorship.get('is_corresponding')]

    first_authors = [authorship.get('author', {}) for authorship in first_authorships]
    last_authors = [authorship.get('author', {}) for authorship in last_authorships]
    corresponding_authors = [authorship.get('author', {}) for authorship in corresponding_authorships]

    first_author_names = [author.get('display_name', '') for author in first_authors]
    last_author_names = [author.get('display_name', '') for author in last_authors]
    corresponding_author_names = [author.get('display_name', '') for author in corresponding_authors]

    first_authors_countries = [authorship.get('countries', [np.nan])[0] if authorship.get('countries') else np.nan for authorship in first_authorships]
    last_authors_countries = [authorship.get('countries', [np.nan])[0] if authorship.get('countries') else np.nan for authorship in last_authorships]
    corresponding_authors_countries = [authorship.get('countries', [np.nan])[0] if authorship.get('countries') else np.nan for authorship in corresponding_authorships]

    journal = np.nan
    primary_location = work.get('primary_location', {})
    source = primary_location.get('source', {}) if isinstance(primary_location, dict) and 'source' in primary_location else {}

    apc_list = work.get('apc_list', {}) or {}
    apc_paid = work.get('apc_paid', {}) or {}

    # Initialize the citation counts dictionary
    citation_counts = {}
    for count_data in work.get('counts_by_year', []):
        year = count_data.get('year')
        if year:
            citation_counts[f'Citations {year}'] = count_data.get('cited_by_count', 0)

    publication_info = {
        'OpenAlex DOI': work.get('doi'),
        'OpenAlex ID': work.get('id'),
        'OpenAlex Title': work.get('display_name'),
        'OpenAlex Abstract': abstract,
        'OpenAlex Publication Year': work.get('publication_year'),
        'OpenAlex Publication Date': work.get('publication_date'),
        'OpenAlex Authors': ', '.join([authorship['author'].get('display_name', '') for authorship in work.get('authorships', [])]),
        'OpenAlex First Authors': ', '.join(first_author_names),
        'OpenAlex Last Authors': ', '.join(last_author_names),
        'OpenAlex Corresponding Authors': ', '.join(corresponding_author_names),
        'OpenAlex Institutions': ', '.join(institutions_country),
        'OpenAlex Institutions Type': ', '.join(institution_type),
        'OpenAlex First Authors Institutions Countries': ', '.join(first_authors_institutions_countries),
        'OpenAlex Last Authors Institutions Countries': ', '.join(last_authors_institutions_countries),
        'OpenAlex Institutions Distinct Count': work.get('institutions_distinct_count'),
        'OpenAlex Countries': ', '.join(map(str, author_countries)),
        'OpenAlex First Authors Countries': ', '.join(map(str, first_authors_countries)),
        'OpenAlex Last Authors Countries': ', '.join(map(str, last_authors_countries)),
        'OpenAlex Corresponding Authors Countries': ', '.join(map(str, corresponding_authors_countries)),
        'OpenAlex Countries Distinct Count': work.get('countries_distinct_count'),
        'OpenAlex Journal': source['display_name'] if source else np.nan,
        'is_in_doaj': source['is_in_doaj'] if source else np.nan,
        'OpenAlex Crossref Publisher': crossref_metadata.get('publisher', ''),
        'OpenAlex Crossref Subject': ', '.join(crossref_metadata.get('subject', [])),
        'OpenAlex topics': ', '.join([topic.get('display_name') for topic in work.get('topics', [])]),
        'OpenAlex Keywords': ', '.join([keyword.get('display_name') for keyword in work.get('keywords', [])]),
        'OpenAlex Mesh': ', '.join([mesh.get('descriptor_name') for mesh in work.get('mesh', [])]),
        'OpenAlex Concepts': ', '.join([concept.get('display_name') for concept in work.get('concepts', [])]),
        'OpenAlex Sustainable Development Goals': ', '.join([sdg.get('display_name') for sdg in work.get('sustainable_development_goals', [])]),
        'Crossref Citation count': crossref_metadata.get('is-referenced-by-count', 0),
        'OpenAlex Cited by Count': work.get('cited_by_count'),
        'OpenAlex Counts by Year': str(work.get('counts_by_year')),
        'OpenAlex Publication Type': work.get('type'),
        'OpenAlex Indexed In': str(work.get('indexed_in')),
        'OpenAlex Language': work.get('language'),
        'OpenAlex Open Access': work.get('open_access', {}).get('is_oa', None),
        'OpenAlex Open Access status': work.get('open_access', {}).get('oa_status', None),
        'OpenAlex grants': ', '.join([grant.get('funder_display_name') for grant in work.get('grants', [])]),
        'Crossref Funders': ', '.join(funder.get('name', '') for funder in crossref_metadata.get('funder', [])),
        'Altmetric Score': altmetric_data.get('score'),
        'Altmetric Read Count': altmetric_data.get('readers_count'),
        'Altmetric Image': altmetric_data.get('images', {}).get('small'),
        'Altmetric URL': altmetric_data.get('details_url'),
        'apc_list_value': apc_list.get('value', None),
        'apc_list_currency': apc_list.get('currency', None),
        'apc_list_value_usd': apc_list.get('value_usd', None),
        'apc_list_provenance': apc_list.get('provenance', None),
        'apc_paid_value': apc_paid.get('value', None),
        'apc_paid_currency': apc_paid.get('currency', None),
        'apc_paid_value_usd': apc_paid.get('value_usd', None),
        'apc_paid_provenance': apc_paid.get('provenance', None),
        'number_of_authors': len(work.get('authorships', []))
    }

    # Add citation counts to publication_info
    publication_info.update(citation_counts)

    return publication_info

search_query = 'https://api.openalex.org/works?page=1&filter=title_and_abstract.search:bibliometric*+OR+scientometric*,has_abstract:true,type:!types/dataset|types/retraction,publication_year:1940-2020'
#search_query_test = 'https://api.openalex.org/works?page=1&filter=has_abstract:true,type:!types/retraction|types/dataset,display_name.search:bibliometric*+OR+scientometric*,publication_year:2000+-+2001'
publication_data = build_publications_dataframe(search_query, True, True)

# Save the data to various formats
publication_data.to_csv('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_20241013_tiab_1950_2020.csv')
#publication_data.to_excel('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_20241011.xlsx')
publication_data.to_pickle('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_20241013_tiab_1950_2020.pkl')

publication_data

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.4236/ajibm.2016.63029
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.1080/19474199.2010.493855
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.18002/rama.v3i4.387
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.21154/pustakaloka.v9i2.1092
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.22201/iibi.24488321xe.2017.nesp1.57888
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.4324/9780429346187-5
Error fetching Altmetric data: 404 Client Error: Not Found for url: https://api.altmetric.com/v1/doi/10.4314/njhbs.v3i1.11504
Error fetching C

,OpenAlex DOI,OpenAlex ID,OpenAlex Title,OpenAlex Abstract,OpenAlex Publication Year,OpenAlex Publication Date,OpenAlex Authors,OpenAlex First Authors,OpenAlex Last Authors,OpenAlex Corresponding Authors,OpenAlex Institutions,OpenAlex Institutions Type,OpenAlex First Authors Institutions Countries,OpenAlex Last Authors Institutions Countries,OpenAlex Institutions Distinct Count,OpenAlex Countries,OpenAlex First Authors Countries,OpenAlex Last Authors Countries,OpenAlex Corresponding Authors Countries,OpenAlex Countries Distinct Count,OpenAlex Journal,is_in_doaj,OpenAlex Crossref Publisher,OpenAlex Crossref Subject,OpenAlex topics,OpenAlex Keywords,OpenAlex Mesh,OpenAlex Concepts,OpenAlex Sustainable Development Goals,Crossref Citation count,OpenAlex Cited by Count,OpenAlex Counts by Year,OpenAlex Publication Type,OpenAlex Indexed In,OpenAlex Language,OpenAlex Open Access,OpenAlex Open Access status,OpenAlex grants,Crossref Funders,Altmetric Score,Altmetric Read Count,Altmetric Image,Altmetric URL,apc_list_value,apc_list_currency,apc_list_value_usd,apc_list_provenance,apc_paid_value,apc_paid_currency,apc_paid_value_usd,apc_paid_provenance,number_of_authors,Citations 2024,Citations 2023,Citations 2022,Citations 2021,Citations 2020,Citations 2019,Citations 2018,Citations 2017,Citations 2016,Citations 2015,Citations 2014,Citations 2013,Citations 2012,OpenAlex Countries Full Name,OpenAlex Countries Region,OpenAlex Countries Income,OpenAlex First Authors Countries Full Name,OpenAlex First Authors Countries Region,OpenAlex First Authors Countries Income,OpenAlex Last Authors Countries Full Name,OpenAlex Last Authors Countries Region,OpenAlex Last Authors Countries Income,OpenAlex Corresponding Authors Countries Full Name,OpenAlex Corresponding Authors Countries Region,OpenAlex Corresponding Authors Countries Income
0,https://doi.org/10.1007/s11192-009-0146-3,https://openalex.org/W2150220236,"Software survey: VOSviewer, a computer program...","We present VOSviewer, a freely available compu...",2009,2009-12-30,"Nees Jan van Eck, Ludo Waltman",Nees Jan van Eck,Ludo Waltman,,Leiden University (NL),Leiden University (education),Leiden University (NL),Leiden University (NL),1,NL,NL,NL,,1,Scientometrics,False,Springer Science and Business Media LLC,,Information Visualization and Visual Data Mini...,"Information Visualization, Interactive Visuali...",,"Computer science, Construct (python library), ...",,9908,12322,"[{'year': 2024, 'cited_by_count': 2487}, {'yea...",article,"['crossref', 'pubmed']",en,True,hybrid,,,87.484,9223.0,https://badges.altmetric.com/?size=64&score=88...,https://www.altmetric.com/details.php?citation...,2290.0,EUR,2890.0,doaj,2290.0,EUR,2890.0,doaj,2,2487.0,3183.0,2441.0,1652.0,991.0,571.0,359.0,172.0,132.0,94.0,78.0,44.0,27.0,Netherlands,Europe,High income,Netherlands,Europe,High income,Netherlands,Europe,High income,,,
1,https://doi.org/10.1177/1094428114562629,https://openalex.org/W3125707221,Bibliometric Methods in Management and Organiz...,We aim to develop a meaningful single-source r...,2014,2014-12-22,"Ivan Župič, Tomaž Čater",Ivan Župič,Tomaž Čater,Ivan Župič,University of Ljubljana (SI),University of Ljubljana (education),University of Ljubljana (SI),University of Ljubljana (SI),1,SI,SI,SI,SI,1,Organizational Research Methods,False,SAGE Publications,,Knowledge Management and Organizational Innova...,"Bibliographic coupling, Bibliometric Analysis,...",,"Bibliographic coupling, Citation, Bibliometric...",Quality education,3038,3985,"[{'year': 2024, 'cited_by_count': 806}, {'year...",article,['crossref'],en,True,green,,,19.600,4730.0,https://badges.altmetric.com/?size=64&score=20...,https://www.altmetric.com/details.php?citation...,NaN,None,NaN,None,NaN,None,NaN,None,2,806.0,1099.0,861.0,544.0,339.0,169.0,78.0,38.0,19.0,1.0,2.0,1.0,NaN,Slovenia,Europe,High income,Slovenia,Europe,High income,Slovenia,Europe,High income,Slovenia,Europe,High income
2,https://doi.org/10.1002/asi.4630270505,https://openalex.org/W2080

In [ ]:
#publication_data.to_pickle('/content/drive/Othercomputers/My laptop/D:/COGNOS X/RESEARH PROJECT/bibliometric_database/data/bibliometric_database_20241011.pkl')

In [ ]:
publication_data

,OpenAlex DOI,OpenAlex ID,OpenAlex Title,OpenAlex Abstract,OpenAlex Publication Year,OpenAlex Publication Date,OpenAlex Authors,OpenAlex First Authors,OpenAlex Last Authors,OpenAlex Corresponding Authors,OpenAlex Institutions,OpenAlex Institutions Type,OpenAlex First Authors Institutions Countries,OpenAlex Last Authors Institutions Countries,OpenAlex Institutions Distinct Count,OpenAlex Countries,OpenAlex First Authors Countries,OpenAlex Last Authors Countries,OpenAlex Corresponding Authors Countries,OpenAlex Countries Distinct Count,OpenAlex Journal,is_in_doaj,OpenAlex Crossref Publisher,OpenAlex Crossref Subject,OpenAlex topics,OpenAlex Keywords,OpenAlex Mesh,OpenAlex Concepts,OpenAlex Sustainable Development Goals,Crossref Citation count,OpenAlex Cited by Count,OpenAlex Counts by Year,OpenAlex Publication Type,OpenAlex Indexed In,OpenAlex Language,OpenAlex Open Access,OpenAlex Open Access status,OpenAlex grants,Crossref Funders,Altmetric Score,Altmetric Read Count,Altmetric Image,Altmetric URL,apc_list_value,apc_list_currency,apc_list_value_usd,apc_list_provenance,apc_paid_value,apc_paid_currency,apc_paid_value_usd,apc_paid_provenance,number_of_authors,Citations 2024,Citations 2023,Citations 2022,Citations 2021,Citations 2020,Citations 2019,Citations 2018,Citations 2017,Citations 2016,Citations 2015,Citations 2014,Citations 2013,Citations 2012,OpenAlex Countries Full Name,OpenAlex Countries Region,OpenAlex Countries Income,OpenAlex First Authors Countries Full Name,OpenAlex First Authors Countries Region,OpenAlex First Authors Countries Income,OpenAlex Last Authors Countries Full Name,OpenAlex Last Authors Countries Region,OpenAlex Last Authors Countries Income,OpenAlex Corresponding Authors Countries Full Name,OpenAlex Corresponding Authors Countries Region,OpenAlex Corresponding Authors Countries Income
0,https://doi.org/10.1007/s11192-009-0146-3,https://openalex.org/W2150220236,"Software survey: VOSviewer, a computer program...","We present VOSviewer, a freely available compu...",2009,2009-12-30,"Nees Jan van Eck, Ludo Waltman",Nees Jan van Eck,Ludo Waltman,,Leiden University (NL),Leiden University (education),Leiden University (NL),Leiden University (NL),1,NL,NL,NL,,1,Scientometrics,False,Springer Science and Business Media LLC,,Information Visualization and Visual Data Mini...,"Information Visualization, Interactive Visuali...",,"Computer science, Construct (python library), ...",,9908,12322,"[{'year': 2024, 'cited_by_count': 2487}, {'yea...",article,"['crossref', 'pubmed']",en,True,hybrid,,,87.484,9223.0,https://badges.altmetric.com/?size=64&score=88...,https://www.altmetric.com/details.php?citation...,2290.0,EUR,2890.0,doaj,2290.0,EUR,2890.0,doaj,2,2487.0,3183.0,2441.0,1652.0,991.0,571.0,359.0,172.0,132.0,94.0,78.0,44.0,27.0,Netherlands,Europe,High income,Netherlands,Europe,High income,Netherlands,Europe,High income,,,
1,https://doi.org/10.1177/1094428114562629,https://openalex.org/W3125707221,Bibliometric Methods in Management and Organiz...,We aim to develop a meaningful single-source r...,2014,2014-12-22,"Ivan Župič, Tomaž Čater",Ivan Župič,Tomaž Čater,Ivan Župič,University of Ljubljana (SI),University of Ljubljana (education),University of Ljubljana (SI),University of Ljubljana (SI),1,SI,SI,SI,SI,1,Organizational Research Methods,False,SAGE Publications,,Knowledge Management and Organizational Innova...,"Bibliographic coupling, Bibliometric Analysis,...",,"Bibliographic coupling, Citation, Bibliometric...",Quality education,3038,3985,"[{'year': 2024, 'cited_by_count': 806}, {'year...",article,['crossref'],en,True,green,,,19.600,4730.0,https://badges.altmetric.com/?size=64&score=20...,https://www.altmetric.com/details.php?citation...,NaN,None,NaN,None,NaN,None,NaN,None,2,806.0,1099.0,861.0,544.0,339.0,169.0,78.0,38.0,19.0,1.0,2.0,1.0,NaN,Slovenia,Europe,High income,Slovenia,Europe,High income,Slovenia,Europe,High income,Slovenia,Europe,High income
2,https://doi.org/10.1002/asi.4630270505,https://openalex.org/W2080